In [1]:
import dgl
import torch as th
import scipy.sparse as sp
spmat = sp.rand(100, 100, density=0.05) # 5%非零项
dgl.from_scipy(spmat)                   # 来自SciPy
import networkx as nx
nx_g = nx.path_graph(5) # 一条链路0-1-2-3-4
dgl.from_networkx(nx_g) # 来自NetworkX

/home/ycwang/miniconda3/envs/brt_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Graph(num_nodes=5, num_edges=8,
      ndata_schemes={}
      edata_schemes={})

In [2]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
print('Number of categories:', dataset.num_classes)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
Number of categories: 7


In [4]:
g = dataset[0]
print(dataset)
print(len(dataset))
print(g)

Dataset("cora_v2", num_graphs=1, save_path=/home/ycwang/.dgl/cora_v2)
1
Graph(num_nodes=2708, num_edges=10556,
      ndata_schemes={'feat': Scheme(shape=(1433,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'train_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={})


In [5]:
print('Node features')
print(g.ndata)
print(g.ndata['label'])
print(len(g.ndata['label']))
print('Edge features')
print(g.edata)

Node features
{'feat': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), 'label': tensor([3, 4, 4,  ..., 3, 3, 3]), 'test_mask': tensor([False, False, False,  ...,  True,  True,  True]), 'val_mask': tensor([False, False, False,  ..., False, False, False]), 'train_mask': tensor([ True,  True,  True,  ..., False, False, False])}
tensor([3, 4, 4,  ..., 3, 3, 3])
2708
Edge features
{}


In [6]:
print(g.ndata['feat'].shape)

torch.Size([2708, 1433])


In [7]:
from dgl.nn import GraphConv

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        # print(h.shape)
        h = F.relu(h)
        # print(h.shape)
        h = self.conv2(g, h)
        # print(h.shape)
        return h

# Create the model with given dimensions
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)

In [8]:
g.ndata['feat'].shape[1]

1433

In [9]:
def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    print('labels ',labels)
    for e in range(1000):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))
model = GCN(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

labels  tensor([3, 4, 4,  ..., 3, 3, 3])
In epoch 0, loss: 1.946, val acc: 0.128 (best 0.128), test acc: 0.142 (best 0.142)
In epoch 5, loss: 1.893, val acc: 0.586 (best 0.586), test acc: 0.620 (best 0.620)
In epoch 10, loss: 1.812, val acc: 0.612 (best 0.612), test acc: 0.653 (best 0.653)
In epoch 15, loss: 1.705, val acc: 0.640 (best 0.640), test acc: 0.672 (best 0.672)
In epoch 20, loss: 1.574, val acc: 0.662 (best 0.662), test acc: 0.701 (best 0.701)
In epoch 25, loss: 1.421, val acc: 0.690 (best 0.690), test acc: 0.703 (best 0.703)
In epoch 30, loss: 1.252, val acc: 0.702 (best 0.702), test acc: 0.716 (best 0.716)
In epoch 35, loss: 1.076, val acc: 0.718 (best 0.720), test acc: 0.732 (best 0.726)
In epoch 40, loss: 0.903, val acc: 0.726 (best 0.726), test acc: 0.747 (best 0.747)
In epoch 45, loss: 0.743, val acc: 0.734 (best 0.734), test acc: 0.752 (best 0.752)
In epoch 50, loss: 0.602, val acc: 0.738 (best 0.738), test acc: 0.758 (best 0.757)
In epoch 55, loss: 0.484, val acc: 0.